# 3. TrainPredict_FineTune
Kaggle score: 0.18203

Tensorboard:
- Input at command: tensorboard --logdir=./log
- Input at browser: http://127.0.0.1:6006

Reference:
- https://www.kaggle.com/codename007/a-very-extensive-landmark-exploratory-analysis

## Run name

In [1]:
import time

project_name = 'ic_furniture2018'
step_name = 'TrainPredict_FineTune'
time_str = time.strftime("%Y%m%d_%H%M%S", time.localtime())
run_name = project_name + '_' + step_name + '_' + time_str
print('run_name: ' + run_name)
t0 = time.time()

run_name: ic_furniture2018_TrainPredict_FineTune_20180502_064158


## Improtant Params

In [2]:
image_size = 224
width = height = image_size
target_size = (width, height)
batch_size = 32
epochs = 25

## Import PKGs

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from IPython.display import display

import os
import gc
import math
import shutil
import zipfile
import pickle
import h5py
from PIL import Image

from tqdm import tqdm
from multiprocessing import cpu_count

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Project folders

In [4]:
cwd = os.getcwd()
input_folder = os.path.join(cwd, 'input')
output_folder = os.path.join(cwd, 'output')
model_folder = os.path.join(cwd, 'model')
feature_folder = os.path.join(cwd, 'feature')
post_pca_feature_folder = os.path.join(cwd, 'post_pca_feature')
log_folder = os.path.join(cwd, 'log')
print('input_folder: \t\t\t%s' % input_folder)
print('output_folder: \t\t\t%s' % output_folder)
print('model_folder: \t\t\t%s' % model_folder)
print('feature_folder: \t\t%s' % feature_folder)
print('post_pca_feature_folder: \t%s' % post_pca_feature_folder)
print('log_folder: \t\t\t%s' % log_folder)

org_train_folder = os.path.join(input_folder, 'org_train')
org_val_folder = os.path.join(input_folder, 'org_val')
org_test_folder = os.path.join(input_folder, 'org_test')
train_folder = os.path.join(input_folder, 'data_train')
val_folder = os.path.join(input_folder, 'data_val')
test_folder = os.path.join(input_folder, 'data_test')
test_sub_folder = os.path.join(test_folder, 'test')

if not os.path.exists(post_pca_feature_folder):
    os.mkdir(post_pca_feature_folder)
    print('Create folder: %s' % post_pca_feature_folder)

train_json_file = os.path.join(input_folder, 'train.json')
val_json_file = os.path.join(input_folder, 'validation.json')
test_json_file = os.path.join(input_folder, 'test.json')
print('\ntrain_json_file: \t\t%s' % train_json_file)
print('val_json_file: \t\t\t%s' % val_json_file)
print('test_json_file: \t\t%s' % test_json_file)

train_csv_file = os.path.join(input_folder, 'train.csv')
val_csv_file = os.path.join(input_folder, 'validation.csv')
test_csv_file = os.path.join(input_folder, 'test.csv')
print('\ntrain_csv_file: \t\t%s' % train_csv_file)
print('val_csv_file: \t\t\t%s' % val_csv_file)
print('test_csv_file: \t\t\t%s' % test_csv_file)

sample_submission_csv_file = os.path.join(input_folder, 'sample_submission_randomlabel.csv')
print('\nsample_submission_csv_file: \t%s' % sample_submission_csv_file)

input_folder: 			/data1/kaggle/imaterialist-challenge-furniture-2018/input
output_folder: 			/data1/kaggle/imaterialist-challenge-furniture-2018/output
model_folder: 			/data1/kaggle/imaterialist-challenge-furniture-2018/model
feature_folder: 		/data1/kaggle/imaterialist-challenge-furniture-2018/feature
post_pca_feature_folder: 	/data1/kaggle/imaterialist-challenge-furniture-2018/post_pca_feature
log_folder: 			/data1/kaggle/imaterialist-challenge-furniture-2018/log

train_json_file: 		/data1/kaggle/imaterialist-challenge-furniture-2018/input/train.json
val_json_file: 			/data1/kaggle/imaterialist-challenge-furniture-2018/input/validation.json
test_json_file: 		/data1/kaggle/imaterialist-challenge-furniture-2018/input/test.json

train_csv_file: 		/data1/kaggle/imaterialist-challenge-furniture-2018/input/train.csv
val_csv_file: 			/data1/kaggle/imaterialist-challenge-furniture-2018/input/validation.csv
test_csv_file: 			/data1/kaggle/imaterialist-challenge-furniture-2018/input/test.csv


## Preview data

In [5]:
train_csv = pd.read_csv(train_csv_file)
print('train_csv.shape is {0}.'.format(train_csv.shape))
display(train_csv.head(2))

val_csv = pd.read_csv(val_csv_file)
print('val_csv.shape is {0}.'.format(val_csv.shape))
display(val_csv.head(2))

test_csv = pd.read_csv(test_csv_file)
print('test_csv.shape is {0}.'.format(test_csv.shape))
display(test_csv.head(2))

test_csv = pd.read_csv(test_csv_file)
print('test_csv.shape is {0}.'.format(test_csv.shape))
display(test_csv.head(2))

sample_submission_csv = pd.read_csv(sample_submission_csv_file)
print('sample_submission_csv.shape is {0}.'.format(sample_submission_csv.shape))
display(sample_submission_csv.head(2))

train_csv.shape is (194828, 3).


,image_id,label_id,url
0,1,5,https://img13.360buyimg.com/imgzone/jfs/t2857/...
1,2,5,http://www.tengdakeli.cn/350/timg01/uploaded/i...


val_csv.shape is (6400, 3).


,image_id,label_id,url
0,1,38,http://www.ghs.net/public/images/fb/3d/51/3beb...
1,2,63,https://img.alicdn.com/imgextra/TB2chFei9YH8KJ...


test_csv.shape is (12800, 2).


,image_id,url
0,1,https://img13.360buyimg.com/imgzone/jfs/t13174...
1,2,http://img35.ddimg.cn/79/22/1258168705-1_u.jpg


test_csv.shape is (12800, 2).


,image_id,url
0,1,https://img13.360buyimg.com/imgzone/jfs/t13174...
1,2,http://img35.ddimg.cn/79/22/1258168705-1_u.jpg


sample_submission_csv.shape is (12800, 2).


,id,predicted
0,1,57
1,2,74


In [6]:
train_id = train_csv['image_id']
train_label_id = train_csv['label_id']

id_2_train_label_id_dict = dict(zip(train_id, train_label_id))
print('len(id_2_train_label_id_dict)=%d' % len(id_2_train_label_id_dict))

index = 0
print('id: %s, \tlandmark_id:%s' % (train_id[index], id_2_train_label_id_dict[train_id[index]]))
index = 1
print('id: %s, \tlandmark_id:%s' % (train_id[index], id_2_train_label_id_dict[train_id[index]]))

image_file = '%s_%s.jpg' % (train_id[index], id_2_train_label_id_dict[train_id[index]])
print(image_file)

len(id_2_train_label_id_dict)=194828
id: 1, 	landmark_id:5
id: 2, 	landmark_id:5
2_5.jpg


In [7]:
val_id = val_csv['image_id']
val_label_id = val_csv['label_id']

id_2_val_label_id_dict = dict(zip(val_id, val_label_id))
print('len(id_2_val_label_id_dict)=%d' % len(id_2_val_label_id_dict))

index = 0
print('id: %s, \tlandmark_id:%s' % (val_id[index], id_2_val_label_id_dict[val_id[index]]))
index = 1
print('id: %s, \tlandmark_id:%s' % (val_id[index], id_2_val_label_id_dict[val_id[index]]))

image_file = '%s_%s.jpg' % (val_id[index], id_2_val_label_id_dict[val_id[index]])
print(image_file)

len(id_2_val_label_id_dict)=6400
id: 1, 	landmark_id:38
id: 2, 	landmark_id:63
2_63.jpg


In [8]:
test_id = test_csv['image_id']

index = 0
print('id: %s' % (test_id[index]))
index = 1
print('id: %s' % (test_id[index]))

image_file = '%s.jpg' % (test_id[index])
print(image_file)

id: 1
id: 2
2.jpg


## Data Generator

In [9]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler, TensorBoard, EarlyStopping

Using TensorFlow backend.


In [10]:
train_gen = ImageDataGenerator(
#     rotation_range=20,
    height_shift_range=0.1,
    width_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
#     channel_shift_range=20,
#     horizontal_flip=True,
#     vertical_flip=True,
    fill_mode='wrap'
)

val_gen = ImageDataGenerator()
test_gen = ImageDataGenerator()

In [11]:
data_train_path = os.path.join(input_folder, 'data_train')
data_val_path = os.path.join(input_folder, 'data_val')
data_test_path = os.path.join(input_folder, 'data_test')

In [12]:
train_generator = train_gen.flow_from_directory(data_train_path, target_size, shuffle=True, batch_size=batch_size)
val_generator = val_gen.flow_from_directory(data_val_path, target_size, shuffle=False, batch_size=batch_size)
test_generator = test_gen.flow_from_directory(data_test_path, target_size, shuffle=False, batch_size=batch_size)

Found 191261 images belonging to 128 classes.
Found 6301 images belonging to 128 classes.
Found 12652 images belonging to 1 classes.


In [13]:
print(train_generator.classes[:10])
print(val_generator.classes[:10])
print(test_generator.classes[:10])

print(test_generator.filenames[:10])

[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
['test/1.jpg', 'test/10.jpg', 'test/100.jpg', 'test/1000.jpg', 'test/10000.jpg', 'test/10001.jpg', 'test/10002.jpg', 'test/10003.jpg', 'test/10004.jpg', 'test/10005.jpg']


In [14]:
steps_per_epoch_train = int(len(train_generator.classes) / batch_size)
steps_per_epoch_val = int(len(val_generator.classes) / batch_size)
print('steps_per_epoch_train=%s' % steps_per_epoch_train)
print('steps_per_epoch_val=%s' % steps_per_epoch_val)

steps_per_epoch_train=5976
steps_per_epoch_val=196


## Build NN

In [15]:
from sklearn.metrics import confusion_matrix

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.applications import *

In [16]:
%%time

input_tensor = Input((height, width, 3))
input_tensor_preprocess = Lambda(inception_resnet_v2.preprocess_input)(input_tensor)
base_model = InceptionResNetV2(input_tensor=input_tensor_preprocess, weights='imagenet', include_top=False)
x = GlobalAveragePooling2D()(base_model.output)

x = Dropout(0.2)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(1024, activation='relu')(x)
x = Dense(128, activation='softmax')(x)

model = Model(inputs=input_tensor, outputs=x)

for layer in base_model.layers:
    layer.trainable = True

CPU times: user 34.6 s, sys: 0 ns, total: 34.6 s
Wall time: 33.9 s


In [17]:
%%time
model.compile(
    optimizer=Adam(lr=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 111, 111, 32) 864         lambda_1[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 111, 111, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation

__________________________________________________________________________________________________
batch_normalization_181 (BatchN (None, 5, 5, 192)    576         conv2d_181[0][0]                 
__________________________________________________________________________________________________
activation_181 (Activation)     (None, 5, 5, 192)    0           batch_normalization_181[0][0]    
__________________________________________________________________________________________________
conv2d_182 (Conv2D)             (None, 5, 5, 224)    129024      activation_181[0][0]             
__________________________________________________________________________________________________
batch_normalization_182 (BatchN (None, 5, 5, 224)    672         conv2d_182[0][0]                 
__________________________________________________________________________________________________
activation_182 (Activation)     (None, 5, 5, 224)    0           batch_normalization_182[0][0]    
__________

In [18]:
def get_lr(x):
#     lr = round(3e-4 * 0.96 ** x, 6)
#     if lr < 1e-5:
#         lr = 1e-5
    
    if x <= 13:
        lr = 3e-4
    elif x > 13 and x <= 22:
        lr = 1e-4
    elif x > 22 and x <= 25:
        lr = 5e-5
    else:
        lr = 5e-5
    print(lr, end='  ')
    return lr

for i in range(1, epochs + 1):
    get_lr(i)
    print(i)
# annealer = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)
annealer = LearningRateScheduler(get_lr)

early_stop = EarlyStopping(monitor='acc', min_delta=0.01, patience=0.005, verbose=0, mode='auto')

log_dir = os.path.join(log_folder, run_name)
print('\nlog_dir: %s' % log_dir)
tensorBoard = TensorBoard(log_dir=log_dir)


callbacks = []
callbacks = [annealer]
# callbacks = [annealer]

0.0003  1
0.0003  2
0.0003  3
0.0003  4
0.0003  5
0.0003  6
0.0003  7
0.0003  8
0.0003  9
0.0003  10
0.0003  11
0.0003  12
0.0003  13
0.0001  14
0.0001  15
0.0001  16
0.0001  17
0.0001  18
0.0001  19
0.0001  20
0.0001  21
0.0001  22
5e-05  23
5e-05  24
5e-05  25

log_dir: /data1/kaggle/imaterialist-challenge-furniture-2018/log/ic_furniture2018_TrainPredict_FineTune_20180502_064158


In [19]:
cpu_amount = cpu_count()
print(cpu_amount)

8


In [ ]:
%%time
hist = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_train,
    epochs=epochs, #Increase this when not on Kaggle kernel
    verbose=1,  #1 for ETA, 0 for silent
    callbacks=callbacks,
    max_queue_size=batch_size,
    workers=cpu_amount,
    use_multiprocessing=False,
    validation_data=val_generator,
    validation_steps=steps_per_epoch_val
)

Epoch 1/25
5976/5976 [==============================] - 2133s 357ms/step - loss: 1.3492 - acc: 0.6357 - val_loss: 1.0461 - val_acc: 0.7049
Epoch 2/25
5976/5976 [==============================] - 2075s 347ms/step - loss: 0.8650 - acc: 0.7566 - val_loss: 0.9462 - val_acc: 0.7290
Epoch 3/25
5976/5976 [==============================] - 2072s 347ms/step - loss: 0.7163 - acc: 0.7940 - val_loss: 0.9101 - val_acc: 0.7478
Epoch 4/25
5976/5976 [==============================] - 2072s 347ms/step - loss: 0.6177 - acc: 0.8183 - val_loss: 0.8233 - val_acc: 0.7693
Epoch 5/25
5976/5976 [==============================] - 2074s 347ms/step - loss: 0.5402 - acc: 0.8389 - val_loss: 0.8029 - val_acc: 0.7730
Epoch 6/25
5976/5976 [==============================] - 2073s 347ms/step - loss: 0.4804 - acc: 0.8553 - val_loss: 0.7406 - val_acc: 0.7857
Epoch 7/25
5976/5976 [==============================] - 2075s 347ms/step - loss: 0.4342 - acc: 0.8676 - val_loss: 0.7210 - val_acc: 0.7989
Epoch 8/25
5976/5976 [=====

In [ ]:
%%time
final_loss, final_acc = model.evaluate_generator(
    val_generator,
    max_queue_size=batch_size,
    workers=cpu_amount,
    use_multiprocessing=False,
    steps=steps_per_epoch_val
)
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))

In [ ]:
run_name_acc = run_name + '_' + str(int(final_acc*10000)).zfill(4)
print(run_name_acc)

In [ ]:
histories = pd.DataFrame(hist.history)
histories['epoch'] = hist.epoch
print(histories.columns)
histories_file = os.path.join(model_folder, run_name_acc + '.csv')
histories.to_csv(histories_file, index=False)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(hist.history['loss'], color='b')
plt.plot(hist.history['val_loss'], color='r')
plt.show()
plt.plot(hist.history['acc'], color='b')
plt.plot(hist.history['val_acc'], color='r')
plt.show()

In [ ]:
def save_network(model, run_name):
    cwd = os.getcwd()
    modelPath = os.path.join(cwd, 'model')
    if not os.path.isdir(modelPath):
        os.mkdir(modelPath)
    weigthsFile = os.path.join(modelPath, run_name + '.h5')
    model.save(weigthsFile)
save_network(model, run_name_acc)

### Change Optimizer

In [ ]:
# model.compile(
#     optimizer='sgd',
#     loss='categorical_crossentropy',
#     metrics=['accuracy']
# )

# callbacks = []

In [ ]:
# %%time
# hist = model.fit_generator(train_generator,
#     steps_per_epoch=steps_per_epoch_train,
#     epochs=10, #Increase this when not on Kaggle kernel
#     verbose=1,  #1 for ETA, 0 for silent
#     callbacks=callbacks,
#     max_queue_size=batch_size,
#     workers=cpu_amount,
#     use_multiprocessing=False,
#     validation_data=val_generator,
#     validation_steps=steps_per_epoch_val)

In [ ]:
# %%time
# final_loss, final_acc = model.evaluate_generator(
#     val_generator,
#     max_queue_size=batch_size,
#     workers=cpu_amount,
#     use_multiprocessing=False,
#     steps=steps_per_epoch_val
# )
# print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))

In [ ]:
# run_name_acc = run_name + '_' + str(int(final_acc*10000)).zfill(4)
# print(run_name_acc)

In [ ]:
# histories = pd.DataFrame(hist.history)
# histories['epoch'] = hist.epoch
# print(histories.columns)
# histories_file = os.path.join(model_folder, run_name_acc + '.csv')
# histories.to_csv(histories_file, index=False)

In [ ]:
# import matplotlib.pyplot as plt
# %matplotlib inline

# plt.plot(hist.history['loss'], color='b')
# plt.plot(hist.history['val_loss'], color='r')
# plt.show()
# plt.plot(hist.history['acc'], color='b')
# plt.plot(hist.history['val_acc'], color='r')
# plt.show()

In [ ]:
# def save_network(model, run_name):
#     cwd = os.getcwd()
#     modelPath = os.path.join(cwd, 'model')
#     if not os.path.isdir(modelPath):
#         os.mkdir(modelPath)
#     weigthsFile = os.path.join(modelPath, run_name + '.h5')
#     model.save(weigthsFile)
# save_network(model, run_name_acc)

## Predict

In [ ]:
train_generator = train_gen.flow_from_directory(data_train_path, target_size, shuffle=False, batch_size=batch_size)
train_generator.classes[:10]

In [ ]:
%%time
y_train_proba = model.predict_generator(
    train_generator,
    max_queue_size=128,
    workers=cpu_amount,
    use_multiprocessing=False
)
print(y_train_proba.shape)

In [ ]:
%%time
y_val_proba = model.predict_generator(
    val_generator,
    max_queue_size=128,
    workers=cpu_amount,
    use_multiprocessing=False
)
print(y_val_proba.shape)

In [ ]:
%%time
y_test_proba = model.predict_generator(
    test_generator,
    max_queue_size=128,
    workers=cpu_amount,
    use_multiprocessing=False
)
print(y_test_proba.shape)

In [ ]:
# 这里证明os.listdir()得到的图片名称list不正确
files = os.listdir(os.path.join(test_folder, 'test'))
print(files[:10])

# 这里证明ImageDataGenerator()得到的图片名称list才是正确
# gen = ImageDataGenerator()
# image_size = (299, 299)
# test_generator  = gen.flow_from_directory(test_folder, image_size, shuffle=False, batch_size=batch_size)
print('test_generator')
print(len(test_generator.filenames))
print(test_generator.filenames[:10])

In [ ]:
def save_proba(y_train_proba, y_train, y_val_proba, y_val, y_test_proba, test_filenames, file_name):
    test_filenames = [n.encode('utf8') for n in test_filenames]
    print(test_filenames[:10])
    if os.path.exists(file_name):
        os.remove(file_name)
        print('File removed: \t%s' % file_name)
    with h5py.File(file_name) as h:
        h.create_dataset('y_train_proba', data=y_train_proba)
        h.create_dataset('y_train', data=y_train)
        h.create_dataset('y_val_proba', data=y_val_proba)
        h.create_dataset('y_val', data=y_val)
        h.create_dataset('y_test_proba', data=y_test_proba)
        h.create_dataset('test_filenames', data=test_filenames)
    print('File saved: \t%s' % file_name)

def load_proba(file_name):
    with h5py.File(file_name, 'r') as h:
        y_train_proba = np.array(h['y_train_proba'])
        y_train = np.array(h['y_train'])
        y_val_proba = np.array(h['y_val_proba'])
        y_val = np.array(h['y_val'])
        y_test_proba = np.array(h['y_test_proba'])
        test_filenames = np.array(h['test_filenames'])
    print('File loaded: \t%s' % file_name)
    test_filenames = [n.decode('utf8') for n in test_filenames]
    print(test_filenames[:10])
    
    return y_train_proba, y_train, y_val_proba, y_val, y_test_proba, test_filenames


y_proba_file = os.path.join(model_folder, 'proba_%s.p' % run_name_acc)
save_proba(y_train_proba, train_generator.classes, y_val_proba, val_generator.classes, y_test_proba, test_generator.filenames, y_proba_file)
y_train_proba, y_train, y_val_proba, y_val, y_test_proba, test_filenames = load_proba(y_proba_file)

print(y_train_proba.shape)
print(y_train.shape)
print(y_val_proba.shape)
print(y_val.shape)
print(y_test_proba.shape)
print(len(test_filenames))

In [ ]:
%%time
max_indexes = np.argmax(y_test_proba, -1)
print(max_indexes.shape)

test_dict = {}
for pair in zip(test_generator.filenames, max_indexes):
    image_name, indx = pair[0], int(pair[1])
    image_name = image_name.split('/')[-1]
    image_id = int(image_name.split('.')[0])
#     print(pair[0], image_name, image_id, indx, indx+1, type(image_id), type(indx))
    test_dict[image_id] = indx + 1

#确认图片的id是否能与ImageDataGenerator()对应上
for name in test_generator.filenames[:10]:
    image_name = name.split('/')[-1]
    image_id = int(image_name.split('.')[0])
#     print('%s\t%s\t%s' % (name, image_id, test_dict[image_id]))

In [ ]:
display(sample_submission_csv.head(2))

In [ ]:
%%time
len_sample_submission_csv = len(sample_submission_csv)
print('len(len_sample_submission_csv)=%d' % len_sample_submission_csv)
count = 0
for i in range(len_sample_submission_csv):
    image_id = int(sample_submission_csv.iloc[i, 0])
    if image_id in test_dict:
        pred_label = test_dict[image_id]
#         print('%s\t%s' % (image_id, pred_label))
        sample_submission_csv.iloc[i, 1] = pred_label
    else:
#         print('%s\t%s' % (image_id, 20))
        sample_submission_csv.iloc[i, 1] = 20 # 属于20的类最多，所以全都设置成这个类，可能会比设置成其他得到的结果好
    count += 1
    if count % 1000 == 0:
        print(int(count/1000), end=' ')
display(sample_submission_csv.head(2))

In [ ]:
print(list(set(sample_submission_csv['predicted'])))

In [ ]:
pred_file = os.path.join(output_folder, 'pred_' + run_name_acc + '.csv')
sample_submission_csv.to_csv(pred_file, index=None)

In [ ]:
print(run_name_acc)

t1 = time.time()
print('time cost: %.2f s' % (t1-t0))
print('Done!')